In [37]:
import numpy as np
import networkx as nx
import time
from numba import jit

In [93]:
from tsp_rol import TSPR

In [38]:
def create_model(path):
    """
        Create Graph representation and MDP variables with number of states designated
        path: path of instance
    """
    data_format = path.split('/')[1]
    
    if data_format == 'edge_explicity':
        return read_edge_explicity_instances(path)

    elif data_format == 'node_coordinate':
        return read_node_coordinate_instances(path)
    else:
        raise Exception

In [57]:
def read_edge_explicity_instances(path):
    '''
        Read explicity edge instance
        path: path of instance
        ex:

        5   			 <- number of nodes
        1231 232 12 122  <- edges (0,1)(0,2)(0,3)(0,4)
        336 1161 717 	 <- edges (1,2)(1,3)(1,4)
        848 323			 <- edges (2,3)(2,4)
        212				 <- edges (3,4)

        return: Graph with weights
    '''
    with open(path) as reader:
        # # Complete graph
        G = nx.complete_graph(int(reader.readline()))
        for i in range(G.number_of_nodes()):
            # Read lines to set weights
            nbrs_i = [int(x) for x in reader.readline().split()]
            for j, w in enumerate(nbrs_i, i + 1):
                G.edges[i, j]['w'] = w
                G.edges[j, i]['w'] = w
    return G

In [58]:
def read_node_coordinate_instances(path):
    '''
        Read node coordinate instances
        path: path of instance
        ex:

        5     <- number of nodes
        1 1   <- node 0, coordinate (x, y) = (1, 1) 
        2 3   <- node 0, coordinate (x, y) = (2, 3) 	
        4 5   <- node 0, coordinate (x, y) = (4, 5) 
        3 3   <- node 0, coordinate (x, y) = (3, 3) 
        7 5   <- node 0, coordinate (x, y) = (7, 5) 

        return: Graph with weights
    '''
    with open(path) as reader:
        G = nx.complete_graph(int(reader.readline()))
        X = []
        Y = []
        for _ in range(G.number_of_nodes()):
            x, y = reader.readline().split()
            X.append(float(x))
            Y.append(float(y))
        coordinates = np.array(list(zip(X, Y)))
        dist_matrix = squareform(pdist(coordinates))
        for i in range(G.number_of_nodes()):
            for j in range(G.number_of_nodes()):
                if i != j:
                    G.edges[i, j]['w'] = dist_matrix[i, j]
                    G.edges[j, i]['w'] = dist_matrix[j, i]
    return G

In [82]:
@jit(nopython=True)
def update_adj_mat(tour):
    '''
        Adjacency matrix to nn algorithm purposes
        tour: partial ou complete tour to create adjacency matrix, ex:
        [0, 2, 4]  < for 5 cities [0, 1, 2, 3, 4, 5] and starting city is 0 
        neighbor with travel cost equal inf is never a good option, 
        so cities in tour recieve this travel cost.
        return: adjacency matrix
    '''
    # Adjacency matrix
    #adj_mat = np.array(nx.adjacency_matrix(G, weight='w').todense(), dtype=float)
    adj_mat = ADJ_MAT.copy()
    # set main diagonal to inf
    for i in range(NUM_CITIES): adj_mat[i][i] = np.inf
    # set elements of current tour to inf
    for i in range(NUM_CITIES):
        for t in tour[:-1]:
            adj_mat[i][t] = np.inf
    return adj_mat

In [104]:
@jit(nopython=True)
def calculate_cost(tour):
    '''
        Calc cost of tour.
        tour: tour of traveler, ex:
        [0, 2, 4, 1, 3, 0]  <- for 5 cities and starting city is 0 
        return: cost of tour
    '''
    cost = 0
    for i in range(NUM_CITIES):
        cost += ADJ_MAT[tour[i]][tour[i+1]]
    return cost

In [87]:
@jit(nopython=True)
def nearest_neighbor(tour):
    '''
        Base heuristic to rollout algorithm, the main difference of traditional 
        nearest neighbor algorithm is the beginning  with a partial tour (solution), ex:
        tour is list of nodes [0, 1, 2], for 5 cities and starting city is 0,
        the return is a complete tour, in the form [0, 1, 2, 4, 3, 0] and cost of tour 
        
    '''
    adj_mat = update_adj_mat(tour)
    # Complete tour 
    for _ in range(NUM_CITIES - len(tour)):
        min_index = np.argmin(adj_mat[tour[-1]])
        for t in tour:
            adj_mat[min_index][t] = np.inf
            adj_mat[t][min_index] = np.inf
        tour.append(min_index)
    tour.append(tour[0])
    # Return complete tour
    return tour

In [107]:
def rollout_algorithm(G, starting_node):
    tour = [starting_node]

    for i in range(NUM_CITIES - 1):
        current_tour = tour.copy()
        best_rollout_cost = np.inf
        best_next_city = None
        
        for j in set(G[tour[-1]]) - set(tour):
            current_tour.append(j)

            nn_path = nearest_neighbor(current_tour.copy())
            rollout_cost = calculate_cost(nn_path)
            if rollout_cost < best_rollout_cost:
                best_rollout_cost = rollout_cost
                best_next_city = j
            
            current_tour.pop()
        tour.append(best_next_city)
    tour.append(starting_node)

    return tour, calculate_cost(tour)

In [108]:
# globals
G = create_model('instances/edge_explicity/brazil58.tsp')
ADJ_MAT = np.array(nx.adjacency_matrix(G, weight='w').todense(), dtype=float)
NUM_CITIES = G.number_of_nodes()

In [94]:
exp = TSPR('instances/edge_explicity/brazil58.tsp')

In [102]:
start = time.time()
for i in range(1000):
    exp.nearest_neighbor([0])
end = time.time()

print('Time elapsed: ', end - start)

Time elapsed:  6.074638366699219


In [103]:
# With numba
start = time.time()
for i in range(1000):
    nearest_neighbor([0])
end = time.time()

print('Time elapsed: ', end - start)

Time elapsed:  0.02701735496520996
